<a href="https://colab.research.google.com/github/aee4/MedGemma/blob/main/Finetuning_with_text_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from huggingface_hub import login
login()

In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM

model_name = "google/medgemma-4b-pt"  # pretrained MedGemma
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    torch_dtype="auto"
)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.16M [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.69M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/33.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/35.0 [00:00<?, ?B/s]

...

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.64G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.96G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/133 [00:00<?, ?B/s]

In [3]:
from google.colab import files
uploaded = files.upload()



Saving ECZEMA TEXT DATASET.csv to ECZEMA TEXT DATASET.csv


In [4]:
import pandas as pd
from datasets import Dataset

# Load your uploaded CSV file
data = pd.read_csv("ECZEMA TEXT DATASET.csv")

# Display sample rows
print(data.head(50))

# Convert to Hugging Face Dataset format
dataset = Dataset.from_pandas(data)

# Optional: split into train/test sets
dataset = dataset.train_test_split(test_size=0.1)
train_dataset = dataset["train"]
eval_dataset = dataset["test"]


   Disease name                                               Text
0        eczema  Lately, I've been feeling my elbows becoming c...
1        eczema  I keep getting my scalp becoming scaly and rou...
2        eczema  He says severe soreness on the forearms. Sympt...
3        eczema  Visible signs include a patch of reddened skin...
4        eczema  The patient reports persistent tightness on th...
5        eczema  Inspection reveals a patch of discolored skin ...
6        eczema  Clinically noted: a patch of discolored skin o...
7        eczema  Clinically noted: localized flaky skin on the ...
8        eczema  Patient complains of persistent roughness on t...
9        eczema  I keep getting my around the mouth becoming re...
10       eczema  Observed during consultation: a patch of flaky...
11       eczema  Physical assessment shows localized irritated ...
12       eczema  She mentions intermittent soreness on the fing...
13       eczema  Sometimes my elbows becoming irritated and di

In [5]:
from transformers import AutoTokenizer, AutoModelForCausalLM

model_name = "google/medgemma-4b-pt"  # MedGemma base model

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto")



Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [6]:
!pip install datasets transformers accelerate -q


In [1]:
!pip install -q bitsandbytes transformers peft datasets accelerate


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 MB 12.5 MB/s eta 0:00:00


In [2]:
import pandas as pd
from datasets import Dataset

df = pd.read_csv("ECZEMA TEXT DATASET.csv")
dataset = Dataset.from_pandas(df)

df.head()


,Disease name,Text
0,eczema,"Lately, I've been feeling my elbows becoming c..."
1,eczema,I keep getting my scalp becoming scaly and rou...
2,eczema,He says severe soreness on the forearms. Sympt...
3,eczema,Visible signs include a patch of reddened skin...
4,eczema,The patient reports persistent tightness on th...


In [3]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

model_name = "google/medgemma-4b-pt"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    torch_dtype=torch.float16,
    load_in_8bit=True
)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
`torch_dtype` is deprecated! Use `dtype` instead!
The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [4]:
from peft import LoraConfig, get_peft_model

lora_config = LoraConfig(
    r=8,                      # rank
    lora_alpha=16,            # scaling factor
    target_modules=["q_proj", "v_proj"],  # LoRA layers to adapt (key for memory)
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()


trainable params: 3,223,552 || all params: 4,303,303,024 || trainable%: 0.0749


In [8]:
from datasets import Dataset
from transformers import AutoTokenizer
import pandas as pd

# Load dataset
df = pd.read_csv("ECZEMA TEXT DATASET.csv")

# Convert to Hugging Face dataset
dataset = Dataset.from_pandas(df)

# Load tokenizer
model_name = "google/medgemma-4b-pt"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# 🧠 Preprocessing function (with padding + truncation fix)
def preprocess_function(examples):
    texts = [
        f"Disease: {d}\nDescription: {t}"
        for d, t in zip(examples["Disease name"], examples["Text"])
    ]
    tokenized = tokenizer(
        texts,
        max_length=512,
        truncation=True,
        padding="max_length",  # ✅ ensures consistent lengths
    )
    tokenized["labels"] = tokenized["input_ids"].copy()
    return tokenized

# 🧩 Apply preprocessing with batching
tokenized_datasets = dataset.map(preprocess_function, batched=True)

# ✅ Ready for LoRA fine-tuning step
print("✅ Dataset tokenized successfully and ready for fine-tuning.")


Map:   0%|          | 0/300 [00:00<?, ? examples/s]

✅ Dataset tokenized successfully and ready for fine-tuning.


In [9]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="./medgemma-lora-eczema",
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    num_train_epochs=3,
    learning_rate=2e-4,
    logging_steps=50,
    save_total_limit=2,
    fp16=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets,
    tokenizer=tokenizer,
)


/tmp/ipython-input-1053125851.py:14: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [10]:
trainer.train()



Step,Training Loss
50,0.000000
100,0.000000


TrainOutput(global_step=114, training_loss=0.0, metrics={'train_runtime': 645.7963, 'train_samples_per_second': 1.394, 'train_steps_per_second': 0.177, 'total_flos': 1.00288473808896e+16, 'train_loss': 0.0, 'epoch': 3.0})

In [11]:
model.save_pretrained("./lora-medgemma-eczema-nsoromma-group")


In [14]:
from huggingface_hub import notebook_login

notebook_login()


In [17]:
from huggingface_hub import HfApi

# ✅ Replace with your actual username
repo_id = "aee4/medgemma-lora-eczema-nsoromma-group"

api = HfApi()

# Create repo (won’t fail if it already exists)
api.create_repo(repo_id=repo_id, private=True, exist_ok=True)

# Upload your adapter folder
api.upload_folder(
    folder_path="./medgemma-lora-eczema",  # your LoRA output folder
    repo_id=repo_id,
    repo_type="model"
)

print(f"✅ Uploaded successfully! View it here: https://huggingface.co/{repo_id}")


Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  ...kpoint-114/tokenizer.json: 100%|##########| 33.4MB / 33.4MB            

  ...point-114/tokenizer.model: 100%|##########| 4.69MB / 4.69MB            

  ...eckpoint-114/scheduler.pt: 100%|##########| 1.47kB / 1.47kB            

...

  .../checkpoint-114/scaler.pt:   7%|7         |   100B / 1.38kB            

  ...int-114/training_args.bin:   7%|7         |   422B / 5.84kB            

  ...80456.0c83b2299893.2816.0:   7%|7         |   542B / 7.50kB            

  ...80841.0c83b2299893.2816.1:   7%|7         |   598B / 8.26kB            

✅ Uploaded successfully! View it here: https://huggingface.co/aee4/medgemma-lora-eczema-nsoromma-group
